In [471]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import os

import cleaning

In [470]:
test= False

accounts = cleaning.clean_account()
cards = cleaning.clean_card(test)
clients = cleaning.clean_client()
districts = cleaning.clean_district()
loans = cleaning.clean_loans(test)
trans = cleaning.clean_trans(test)
disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')


D:\outros\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Creation of a new Transactions Dataframe

The new transactions consists on a dataframe where the transactions from the same account are all summarized in 1 row: `min no. trans`, `max no. trans`, `avg no. trans`, `min_balance`, `max_balance` and `avg_balance`.

In [472]:
trans = trans.drop(columns=["trans_id", "date", "type", "operation"])
trans

,account_id,amount,balance
0,5270,800.0,800.0
1,5270,44749.0,45549.0
2,11265,1000.0,1000.0
3,10364,1100.0,1100.0
4,3834,700.0,700.0
...,...,...,...
396680,1763,14.6,67769.5
396681,1765,14.6,19708.1
396682,1775,14.6,15944.5
396683,1769,14.6,34679.4


In [474]:
trans = trans.sort_values(by="account_id")

aux = 0
count = 0
trans_final = pd.DataFrame(columns=["account_id", "no. movements", "min no. trans", "max no. trans", "avg no. trans", "min balance", "max balance", "avg balance"])

for i in loans["account_id"]:
    if i != aux:
        trans_test = trans.loc[trans['account_id'] == i]

        no_movements = trans_test.shape[0]
        min_no_trans = min(trans_test["amount"])
        max_no_trans = max(trans_test["amount"])
        avg_no_trans = mean(trans_test["amount"])
        min_balance = min(trans_test["balance"])
        max_balance = max(trans_test["balance"])
        avg_balance = mean(trans_test["balance"])
        
        trans_final = trans_final.append({"account_id": i, "no. movements": no_movements, "min no. trans": min_no_trans, "max no. trans": max_no_trans, "avg no. trans": avg_no_trans, "min balance": min_balance, "max balance": max_balance, "avg balance": avg_balance}, ignore_index=True)
        
        aux = i

trans_final

,account_id,no. movements,min no. trans,max no. trans,avg no. trans,min balance,max balance,avg balance
0,1787.0,4.0,1100.0,9900.0,5025.000000,1100.0,20100.0,12250.000000
1,1801.0,37.0,2.9,54300.0,11015.635135,700.0,120512.8,52083.859459
2,9188.0,24.0,48.6,19065.0,5417.458333,800.0,49590.4,30060.954167
3,1843.0,25.0,14.6,26448.0,8253.080000,1000.0,65898.5,41297.480000
4,11013.0,27.0,30.0,63366.0,18945.966667,600.0,122893.1,57188.211111
...,...,...,...,...,...,...,...,...
323,9030.0,172.0,14.6,36960.0,4895.974419,200.0,75294.1,44197.509884
324,3189.0,59.0,14.6,52600.0,11725.494915,800.0,130659.1,55230.444068
325,8972.0,39.0,14.6,31636.5,7779.815385,800.0,63659.3,41994.907692
326,10963.0,124.0,14.6,50800.0,13280.084677,1100.0,119527.2,56646.516129


## Client-District Merge

In [475]:
# Add district columns to the right side of the clients columns 
client_district = clients.merge(districts, left_on="district_id", right_on="code ")

# Drop columns that are irrelevant: name of the district, regions of the district and id of the district 
client_district = client_district.drop(columns=["name ", "district_id"])

# Add merge information to columns merged
# First 4 belong to the client, the others belong to the district
client_district = client_district.rename(columns={col:("client_district "+ col) for col in client_district.columns[4:]})

client_district

,client_id,birth_number,gender,code,client_district region,client_district no. of inhabitants,client_district no. of municipalities with inhabitants < 499,client_district no. of municipalities with inhabitants 500-1999,client_district no. of municipalities with inhabitants 2000-9999,client_district no. of municipalities with inhabitants >10000,...,client_district ratio of urban inhabitants,client_district average salary,client_district unemploymant rate '95,client_district unemploymant rate '96,client_district unemploymant_growth,client_district no. of enterpreneurs per 1000 inhabitants,client_district no. of commited crimes '95,client_district no. of commited crimes '96,client_district crime_growth,client_district total_crime
0,1,13-12-1970,female,18,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
1,420,13-03-1978,male,18,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
2,499,08-07-1935,female,18,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
3,519,13-04-1980,male,18,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
4,682,21-10-1979,male,18,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,9930,23-06-1972,male,9,central Bohemia,81344,61,23,4,2,...,55.3,8899,3.39,3.97,0.58,149,2987.0,2487,-500.0,5474.0
5365,10070,23-04-1974,male,9,central Bohemia,81344,61,23,4,2,...,55.3,8899,3.39,3.97,0.58,149,2987.0,2487,-500.0,5474.0
5366,11266,25-09-1938,male,9,central Bohemia,81344,61,23,4,2,...,55.3,8899,3.39,3.97,0.58,149,2987.0,2487,-500.0,5474.0
5367,11267,26-08-1936,female,9,central Bohemia,81344,61,23,4,2,...,55.3,8899,3.39,3.97,0.58,149,2987.0,2487,-500.0,5474.0


## Account-District Merge

In [476]:
# Add district columns to the right side of the accounts columns 
account_district = accounts.merge(districts, left_on="district_id", right_on="code ")

# Drop columns that are irrelevant: name of the district, regions of the district and id of the district 
account_district = account_district.drop(columns=["name ", "district_id", "code "])

# Add merge information to columns merged
# First 4 belong to the client, the others belong to the district
account_district = account_district.rename(columns={col:("account_district "+ col) for col in account_district.columns[4:]})

account_district

,account_id,frequency,date,region,account_district no. of inhabitants,account_district no. of municipalities with inhabitants < 499,account_district no. of municipalities with inhabitants 500-1999,account_district no. of municipalities with inhabitants 2000-9999,account_district no. of municipalities with inhabitants >10000,account_district no. of cities,account_district ratio of urban inhabitants,account_district average salary,account_district unemploymant rate '95,account_district unemploymant rate '96,account_district unemploymant_growth,account_district no. of enterpreneurs per 1000 inhabitants,account_district no. of commited crimes '95,account_district no. of commited crimes '96,account_district crime_growth,account_district total_crime
0,576,monthly issuance,01-01-1993,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,0.55,111,3659.0,3894,235.0,7553.0
1,704,monthly issuance,01-01-1993,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,0.55,111,3659.0,3894,235.0,7553.0
2,192,monthly issuance,08-01-1993,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,0.55,111,3659.0,3894,235.0,7553.0
3,10364,monthly issuance,17-01-1993,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,0.55,111,3659.0,3894,235.0,7553.0
4,497,monthly issuance,15-04-1993,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,0.55,111,3659.0,3894,235.0,7553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,10134,monthly issuance,26-04-1997,west Bohemia,58400,65,16,4,1,6,52.4,8620,1.10,1.25,0.15,100,1089.0,1117,28.0,2206.0
4496,3951,monthly issuance,02-06-1997,west Bohemia,58400,65,16,4,1,6,52.4,8620,1.10,1.25,0.15,100,1089.0,1117,28.0,2206.0
4497,679,monthly issuance,12-07-1997,west Bohemia,58400,65,16,4,1,6,52.4,8620,1.10,1.25,0.15,100,1089.0,1117,28.0,2206.0
4498,2182,monthly issuance,12-07-1997,west Bohemia,58400,65,16,4,1,6,52.4,8620,1.10,1.25,0.15,100,1089.0,1117,28.0,2206.0


## Card-Client Merge

### Useful Renames

In [477]:
# Renamed so it doesn't get any problems while merging with the dispositions
cards = cards.rename(columns={"type":"card_type", "issued":"card_issued"})

cards

,card_id,disp_id,card_type,card_issued
0,1005,9285,classic,07-11-1993
1,104,588,classic,19-01-1994
2,747,4915,classic,05-02-1994
3,70,439,classic,08-02-1994
4,577,3687,classic,15-02-1994
...,...,...,...,...
172,243,1478,classic,13-12-1996
173,162,967,junior,23-12-1996
174,594,3794,junior,27-12-1996
175,609,3893,classic,29-12-1996


### Merging

#### 1. Card-Disposition Merge

In [478]:
# Add disposition columns to the right side of the cards columns 
card_client = cards.merge(disp)

# Drop irrelevant columns: id's except account's id (because of future mergings)
card_client = card_client.drop(columns=["card_id", "disp_id", "client_id", "type"])

card_client

,card_type,card_issued,account_id
0,classic,07-11-1993,7753
1,classic,19-01-1994,489
2,classic,05-02-1994,4078
3,classic,08-02-1994,361
4,classic,15-02-1994,3050
...,...,...,...
172,classic,13-12-1996,1225
173,junior,23-12-1996,807
174,junior,27-12-1996,3138
175,classic,29-12-1996,3219


#### 2. Dataframe of Clients without Cards and Dataframe of Cards without Loans

In [479]:
# Get Clients that have Loans but don't have a Card

loan_client_no_card = []
for i in loans["account_id"]:
    if not i in list(card_client["account_id"]):
        loan_client_no_card.append(i)

# Get Clients that have a Card but don't have Loans

card_client_no_loan = []
for i in card_client["account_id"]:
    if not i in list(loans["account_id"]):
        card_client_no_loan.append(i)

card_client = card_client.drop(index=card_client[card_client["account_id"].isin(card_client_no_loan)].index)

loan_client_no_card = np.array([loan_client_no_card, ["None"] * len(loan_client_no_card), [0] * len(loan_client_no_card)]).transpose()
loan_client_no_card = pd.DataFrame(loan_client_no_card, columns=["account_id", "card_type", "card_issued"])

#### 3. Add to Cards-Client all the informations

In [480]:
card_client = card_client.append(loan_client_no_card)
card_client["account_id"] = card_client["account_id"].astype(int)

card_client

,card_type,card_issued,account_id
0,classic,07-11-1993,7753
4,classic,15-02-1994,3050
43,classic,13-06-1995,11359
67,junior,24-10-1995,4503
73,classic,22-11-1995,11042
...,...,...,...
312,None,0,9030
313,None,0,3189
314,None,0,8972
315,None,0,10963


## Account-Client Merge

In [481]:
# Merge the disposition columns with the accounts (with district information) columns
account_client = disp.merge(account_district)

# Merge the previous table with the clients (with the district information) columns
account_client = account_client.merge(client_district, on="client_id")

# The only loans that are relevant are the ones that belong to the owner
account_client = account_client[account_client["type"] == "OWNER"]

# Drop irrelevant columns: client id and type (it's always OWNER)
account_client = account_client.drop(columns=["client_id", "type"])

#client_district.sort_values(by="client_id")
account_client

,disp_id,account_id,frequency,date,region,account_district no. of inhabitants,account_district no. of municipalities with inhabitants < 499,account_district no. of municipalities with inhabitants 500-1999,account_district no. of municipalities with inhabitants 2000-9999,account_district no. of municipalities with inhabitants >10000,...,client_district ratio of urban inhabitants,client_district average salary,client_district unemploymant rate '95,client_district unemploymant rate '96,client_district unemploymant_growth,client_district no. of enterpreneurs per 1000 inhabitants,client_district no. of commited crimes '95,client_district no. of commited crimes '96,client_district crime_growth,client_district total_crime
0,1,1,monthly issuance,24-03-1995,south Bohemia,70699,60,13,2,1,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
1,2,2,monthly issuance,26-02-1993,Prague,1204953,0,0,0,1,...,100.0,12541,0.29,0.43,0.14,167,85677.0,99107,13430.0,184784.0
3,4,3,monthly issuance,07-07-1997,central Bohemia,95616,65,30,4,1,...,51.4,9307,3.85,4.43,0.58,118,2616.0,3040,424.0,5656.0
5,6,4,monthly issuance,21-02-1996,central Bohemia,107870,84,29,6,1,...,58.0,8754,3.83,4.31,0.48,137,3804.0,3868,64.0,7672.0
6,7,5,monthly issuance,30-05-1997,south Bohemia,58796,22,16,7,1,...,51.9,9045,3.13,3.60,0.47,124,1845.0,1879,34.0,3724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,13623,11333,monthly issuance,26-05-1994,central Bohemia,112065,95,19,7,1,...,69.4,11277,1.25,1.44,0.19,127,5179.0,4987,-192.0,10166.0
5364,13647,11349,weekly issuance,26-05-1995,Prague,1204953,0,0,0,1,...,100.0,12541,0.29,0.43,0.14,167,85677.0,99107,13430.0,184784.0
5366,13660,11359,monthly issuance,01-10-1994,south Moravia,117897,139,28,5,1,...,53.8,8814,4.76,5.74,0.98,107,2112.0,2059,-53.0,4171.0
5367,13663,11362,monthly issuance,14-10-1995,north Moravia,106054,38,25,6,2,...,63.1,8110,5.77,6.55,0.78,109,3244.0,3079,-165.0,6323.0


## Final Merge: Loan

### Useful Rename

In [482]:
loans = loans.rename(columns={"date":"loan_date", "amount":"loan_amount", "duration":"loan_duration"})
trans = trans.rename(columns={"amount":"no. transactions"})
account_client = account_client.rename(columns={"frequency":"account_frequency", "date": "account_creation"})


### Merging

In [483]:
# Merge accounts informations into loans columns
loan_final = loans.merge(account_client, on="account_id")

# Merge cards with clients informations into loans columns
loan_final = loan_final.merge(card_client, on="account_id")

# Merge transactions informations into loans colums
loan_final = loan_final.merge(trans_final, on="account_id")

# Drop irrelevant columns: id's
loan_final


,loan_id,account_id,loan_date,loan_amount,loan_duration,payments,status,disp_id,account_frequency,account_creation,...,client_district total_crime,card_type,card_issued,no. movements,min no. trans,max no. trans,avg no. trans,min balance,max balance,avg balance
0,5314,1787,05-07-1993,96396,12,8033,-1,2166,weekly issuance,22-03-1993,...,5789.0,None,0,4.0,1100.0,9900.0,5025.000000,1100.0,20100.0,12250.000000
1,5316,1801,11-07-1993,165960,36,4610,1,2181,monthly issuance,13-02-1993,...,5472.0,None,0,37.0,2.9,54300.0,11015.635135,700.0,120512.8,52083.859459
2,6863,9188,28-07-1993,127080,60,2118,1,11006,monthly issuance,08-02-1993,...,4202.0,None,0,24.0,48.6,19065.0,5417.458333,800.0,49590.4,30060.954167
3,5325,1843,03-08-1993,105804,36,2939,1,2235,monthly issuance,30-01-1993,...,12899.0,None,0,25.0,14.6,26448.0,8253.080000,1000.0,65898.5,41297.480000
4,7240,11013,06-09-1993,274740,60,4579,1,13231,weekly issuance,14-02-1993,...,3022.0,None,0,27.0,30.0,63366.0,18945.966667,600.0,122893.1,57188.211111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,12-12-1996,155616,48,3242,1,10813,monthly issuance,21-01-1995,...,18880.0,None,0,172.0,14.6,36960.0,4895.974419,200.0,75294.1,44197.509884
324,5625,3189,15-12-1996,222180,60,3703,-1,3855,monthly issuance,29-11-1995,...,1706.0,None,0,59.0,14.6,52600.0,11725.494915,800.0,130659.1,55230.444068
325,6805,8972,21-12-1996,45024,48,938,1,10742,monthly issuance,21-05-1996,...,19986.0,None,0,39.0,14.6,31636.5,7779.815385,800.0,63659.3,41994.907692
326,7233,10963,25-12-1996,115812,36,3217,1,13172,monthly issuance,20-05-1995,...,3787.0,None,0,124.0,14.6,50800.0,13280.084677,1100.0,119527.2,56646.516129
